In [1]:
# ================================================================
#   MAT 531 – Multivariable Mathematics for Machine Learning
#   Chapter 5 – Project B
#
#   Title: Polynomial Root Approximation via Companion Matrices
#   Student: Nikita S. Karim
#   Instructor Professor Rocca
#   Fall 2025
# ================================================================

import numpy as np

def companion_matrix(a):
    """
    Build the companion matrix for a monic polynomial
    p(t) = t^n + a_{n-1} t^{n-1} + ... + a_1 t + a_0

    Parameters
    ----------
    a : list or 1D array
        Coefficients [a0, a1, ..., a_{n-1}].

    Returns
    -------
    C : (n, n) ndarray
        Companion matrix of p.
    """
    a = np.asarray(a, dtype=float)
    n = a.size
    C = np.zeros((n, n), dtype=float)

    # Ones on the subdiagonal
    C[1:, :-1] = np.eye(n - 1)

    # Last column = -a0, -a1, ..., -a_{n-1}
    C[:, -1] = -a
    return C


a1 = [-4, 2]                # [a0, a1]
C1 = companion_matrix(a1)

a2 = [22, 12, -9]           # [a0, a1, a2]
C2 = companion_matrix(a2)

a3 = [24, 14, -13, -2]      # [a0, a1, a2, a3]
C3 = companion_matrix(a3)

print("Companion matrix (a):\n", C1, "\n")
print("Companion matrix (b):\n", C2, "\n")
print("Companion matrix (c):\n", C3, "\n")

#Part (d): verify characteristic polynomials
for i, C in enumerate([C1, C2, C3], start=1):
    coeffs = np.poly(C)   # returns [1, a_{n-1}, ..., a0]
    print(f"Characteristic polynomial coefficients for C{i}: {coeffs}")


Companion matrix (a):
 [[ 0.  4.]
 [ 1. -2.]] 

Companion matrix (b):
 [[  0.   0. -22.]
 [  1.   0. -12.]
 [  0.   1.   9.]] 

Companion matrix (c):
 [[  0.   0.   0. -24.]
 [  1.   0.   0. -14.]
 [  0.   1.   0.  13.]
 [  0.   0.   1.   2.]] 

Characteristic polynomial coefficients for C1: [ 1.  2. -4.]
Characteristic polynomial coefficients for C2: [ 1. -9. 12. 22.]
Characteristic polynomial coefficients for C3: [  1.  -2. -13.  14.  24.]


In [2]:
# ------------------------------------------------
# QR method to approximate eigenvalues (roots)
# ------------------------------------------------

def qr_eigenvalues(A, tol=0.1, max_iter=200):
    """
    QR eigenvalue iteration: A_{k+1} = R_k Q_k

    Stops when all entries below the main diagonal
    have absolute value < tol.

    Parameters
    ----------
    A : (n, n) ndarray
        Initial matrix (companion matrix).
    tol : float
        Tolerance for entries below the diagonal.
    max_iter : int
        Maximum number of iterations.

    Returns
    -------
    A_k : (n, n) ndarray
        Final matrix after QR iterations (almost upper triangular).
    lambdas : 1D ndarray
        Approximate eigenvalues (roots), taken from diag(A_k).
    """
    A_k = A.astype(float).copy()

    for k in range(max_iter):
        Q, R = np.linalg.qr(A_k)
        A_k = R @ Q

        # Check convergence: entries strictly below the diagonal
        below_diag = np.tril(A_k, k=-1)
        if np.all(np.abs(below_diag) < tol):
            print(f"Converged in {k+1} iterations")
            break

    lambdas = np.diag(A_k)
    return A_k, lambdas


# Apply QR method to each companion matrix
for i, C in enumerate([C1, C2, C3], start=1):
    print(f"\n=== Polynomial {i} ===")
    A_final, roots = qr_eigenvalues(C, tol=0.1, max_iter=200)
    print("Final A matrix:\n", A_final)
    print("Approximate roots (diagonal of A_final):\n", roots)


=== Polynomial 1 ===
Converged in 5 iterations
Final A matrix:
 [[-3.27868852 -2.93442623]
 [ 0.06557377  1.27868852]]
Approximate roots (diagonal of A_final):
 [-3.27868852  1.27868852]

=== Polynomial 2 ===
Converged in 6 iterations
Final A matrix:
 [[  6.91708587 -20.90538318  14.55759312]
 [  0.03234061   3.10313866  -1.8767184 ]
 [  0.           0.04375158  -1.02022453]]
Approximate roots (diagonal of A_final):
 [ 6.91708587  3.10313866 -1.02022453]

=== Polynomial 3 ===
Converged in 14 iterations
Final A matrix:
 [[ 3.87534711e+00  1.13683148e+01  2.26676106e+01  1.53111007e+01]
 [ 7.56444776e-02 -2.86638472e+00 -5.29516544e+00 -3.68292938e+00]
 [ 0.00000000e+00 -7.68685647e-03  1.99080684e+00  8.39756579e-01]
 [ 0.00000000e+00  0.00000000e+00  8.13455798e-04 -9.99769226e-01]]
Approximate roots (diagonal of A_final):
 [ 3.87534711 -2.86638472  1.99080684 -0.99976923]
